# Postprocess the response data from the OpenAI API

input: `respose_file_path`
output: `annotated_file_path`

Description
- make the annotated dataset from the response from the OpenAI API.

In [1]:
import pandas as pd
from datasets import Features, ClassLabel, Dataset, DatasetDict
import json
import re

In [2]:
respose_file_path = r'../local_data/api_responses_for_annotated_dataset.jsonl'

In [3]:
in_fname =  respose_file_path
datas = []
request_ids = []
# results = {}
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if isinstance(data[1] , dict):
            # results[i] = data
            datas.append(data[1]['choices'][0]['message']['content'])
            request_ids.append(data[2]['request_id'])

In [4]:
df = pd.DataFrame({'request_id': request_ids, 'response': datas})
df['response'] = df['response'].str.split('\n')
df.sort_values(by='request_id', inplace=True)

# response 컬럼을 펼쳐서 새로운 DataFrame 생성
exploded_df = df.explode('response').reset_index(drop=True)

# "->" 가 포함된 response만 남기기 (한건 제외됨)
exploded_df = exploded_df[exploded_df['response'].str.contains('->')]

In [5]:
# 문자열을 분리하는 함수 정의
def split_response(response):
    # 제목과 나머지를 분리
    title, rest = response.split("->")
    title = title.strip()
    
    # 카테고리와 키워드를 분리
    category, *keywords = rest.split(":")
    category = category.strip()
    keywords = [keyword.strip() for keyword in keywords]
    
    return title, category, keywords


In [6]:
# DataFrame에 새로운 컬럼 추가
exploded_df[['title', 'category', 'keywords']] = exploded_df['response'].apply(lambda x: pd.Series(split_response(x)))


In [7]:
# 필요 없는 response 컬럼 제거
exploded_df = exploded_df.drop(columns=['response'])

# request_id를 'tester'와 'trial_idx', 'sub_idx'로 분리
exploded_df[['tester', 'trial_idx', 'sub_idx']] = exploded_df['request_id'].str.split(':', expand=True)

# trial_idx를 숫자형으로 변환
exploded_df['trial_idx'] = exploded_df['trial_idx'].astype(float).astype(int)

# sub_idx를 숫자형으로 변환
exploded_df['sub_idx'] = exploded_df['sub_idx'].astype(int)

# 필요 없는 request_id 컬럼 제거
exploded_df = exploded_df.drop(columns=['request_id'])


In [10]:
exploded_df.to_csv('../local_data/annotated_dataset.csv', index=False)

----

In [17]:
exploded_df.query('tester.str.contains("유효상")')

,title,category,keywords,tester,trial_idx,sub_idx
1168,"1. 코치가 지시할 수 없는 건데…이정후 보고 배워라美 중계진 극찬, 이종범까지 소...",8.2. 해외 야구,"[이정후, 이종범, 미국 중계진]",유효상,0,0
1169,2. “이렇게 무너질 줄이야” 아무도 몰랐다…넷플릭스발 파산 위기,2.4.1. 글로벌 기업 동향,"[넷플릭스, 파산 위기]",유효상,0,0
1170,"3. 지금 당장 휴지를 냉장고, 싱크대에 넣어보세요, 사장님이 방법을 알려줘 절약하...",4.10. 취미/DIY,"[휴지, 냉장고, 싱크대]",유효상,0,0
1171,4. 인천 개표 지연…윤상현에 밀린 남영희 재검표 요구,1.1.4. 선거,"[인천, 윤상현, 남영희]",유효상,0,0
1172,5. 삼성페이 업그레이드 버전 `삼성월렛` 눈길,5.4. IT 기기,"[삼성페이, 삼성월렛]",유효상,0,0
...,...,...,...,...,...,...
1538,39. 30분 더 일찍 일어나는 부자들… 아침 루틴 봤더니 ‘이것’ 즐겼다,4.1. 건강,"[부자, 아침 루틴]",유효상,7,12
1539,"40. 5연패' 한화, 끝없는 추락…이젠 최하위가 눈앞",8.1. 국내 야구,"[한화, 5연패]",유효상,7,12
1540,41. 정몽규 체제가 낳은 한국 축구 대재앙…40년 공든 탑 무너졌다,8.3. 국내 축구,"[정몽규, 한국 축구]",유효상,7,12
1541,"42. 40년만 올림픽 예선 탈락' 황선홍 감독 ""대표팀 시스템 바뀌어야, 대회 준...",8.3. 국내 축구,"[황선홍, 올림픽 예선]",유효상,7,12


In [15]:
exploded_df.query('tester.str.contains("유효상") and title.str.contains("이정후의 Perfect")')

,title,category,keywords,tester,trial_idx,sub_idx
1351,19. 이정후의 Perfect Day'… 안방 첫 홈런에 + 쐐기 2루타까지 #SP...,8.1. 국내 야구,"[이정후, 홈런, 2루타]",유효상,4,6
